# Temperature, Density and Salinity Profiles for WOD PFL Instruments

This notebook plots temperature, density and salinity profiles for the PFL data from the WOD floats

In [ ]:
# import libraries
import datetime
import numpy
import xarray
import matplotlib.pyplot as plt
%matplotlib inline 
import pandas
import gsw
import cmocean
import sys
import glob
import errno
import os
import os.path
import netCDF4 as nc

In [ ]:
def filecount(dir_name):
    """This function counts number of files in directory provided"""
    return len([f for f in os.listdir(dir_name) if os.path.isfile(f)])

In [ ]:
datadir = '/path/datafolder/' # Insert folder path
for filename in os.listdir(datadir):
    savedir = '/path/savefolder/' 
    # Insert path to folder you'd like to save your plots in
    
    fname = filename
        
    ncdata = xarray.open_dataset(datadir + fname)
    juld = pandas.to_datetime(ncdata.time.values) # convert argument to datetime
        
    press = ncdata.Pressure.values  # pressure (needed for depth)
    temp = ncdata.Temperature.values  # in situ temperature
    salt = ncdata.Salinity.values  # in situ practical salinity
    lats = ncdata.lat.values # latitude
    lons = ncdata.lon.values # longitude

    # calculate variables
    z = gsw.conversions.z_from_p(press, lats)  # depth calculated from pressure
    # find depth of deepest profile point
    SA = gsw.conversions.SA_from_SP(salt, press,
                                        lons, lats)  # absolute salinity
    temp_fp = gsw.t_freezing(SA, press, 0)  # assuming saturation_fraction = 0
    CT = gsw.conversions.CT_from_t(SA, temp, press)  # conservative temperature
    sigma0 = gsw.sigma0(SA, CT)  # density from absolute salinity and conservative temperature
    rho = gsw.density.rho_t_exact(SA, CT, press)  # in situ density
    pot_temp = gsw.conversions.pt0_from_t(SA, temp, press) # potential temperature

    # density profile
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.spines["bottom"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.xaxis.tick_top()
    plt.suptitle('{} [{}]'.format("In-situ Density", "kg/m^3"))
    ax.plot(sigma0, z)
    plt.grid(alpha=0.8, linestyle=':', linewidth=.5)
    plt.savefig(savedir + 'rho_prof_{}_{}'.format(fname[:7], juld.strftime('%Y%m%d_%H%M%S')))
    #plt.close(fig)
            
    # temperature profile
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.spines["bottom"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.xaxis.tick_top()
    plt.suptitle('{} [{}]'.format(ncdata.Temperature.long_name, ncdata.Temperature.units))
    ax.plot(temp, z)
    plt.grid(alpha=0.8, linestyle=':', linewidth=.5)
    plt.savefig(savedir + 'temp_prof_{}_{}'.format(fname[:7], juld.strftime('%Y%m%d_%H%M%S')))
    #plt.close(fig)

    # salinity profile
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.spines["bottom"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.xaxis.tick_top()
    plt.suptitle('{} [{}]'.format(ncdata.Salinity.long_name, "psu"))
    ax.plot(salt, z)
    plt.grid(alpha=0.8, linestyle=':', linewidth=.5)
    plt.savefig(savedir + 'salt_prof_{}_{}'.format(fname[:7], juld.strftime('%Y%m%d_%H%M%S')))
    #plt.close(fig)

    # regular TS diagram: pot temp, practical salinity, sigma0
    te = numpy.linspace(-2, 2, 50)
    se = numpy.linspace(33.8, 34.9, 50)
    plttemp, pltsalt = numpy.meshgrid(te, se)
    pltsigma0 = gsw.sigma0(pltsalt, plttemp) # potential density anomaly wrt ref press of 0 dbar
    cn = numpy.arange(26.5, 39.1, 0.05)

    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_xlim(34.3, 34.8)
    ax.set_ylim(-2, 1)
    cs = plt.contour(pltsalt, plttemp, pltsigma0, linestyles='--', linewidths=.7, colors='grey',
                     levels=cn, zorder=1)
    plt.clabel(cs, fontsize=12, inline=True, fmt='%1.2f')
    sc = ax.scatter(salt, pot_temp, c=z, vmin=-2000., vmax=-0.,
                    cmap=plt.cm.get_cmap('Spectral_r'))
    #                cmap=cmocean.cm.deep)
    cbar = plt.colorbar(sc)
    cbar.ax.set_ylabel('Depth from pressure [m]')
    ax.set_xlabel('Practical salinity')
    ax.set_ylabel('Potential temperature [deg C]')
    plt.title('Float {} at {}'.format(fname[:7], juld.strftime('%Y/%m/%d %H:%M:%S')))
    plt.savefig(savedir + 'TS_pottemp_sigma0_ps_{}_{}'.format(fname[:7], juld.strftime('%Y%m%d_%H%M%S')))
    #plt.close(fig)
            
filecount(os.getcwd()) # Count number of files in folder
plt.rcParams.update({'figure.max_open_warning': 0})

In [ ]:
filename.close()